In [2]:
import featuretools as ft

In [3]:
data = ft.demo.load_mock_customer()

In [4]:
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])
transactions_df.sample(10)

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code,join_date,date_of_birth
264,380,21,2014-01-01 05:14:10,5,57.09,4,desktop,2014-01-01 05:02:15,60091,2011-04-08 20:08:14,2006-08-15
19,244,10,2014-01-01 02:34:55,2,116.95,2,tablet,2014-01-01 02:31:40,13244,2012-04-15 23:31:04,1986-08-18
314,299,6,2014-01-01 01:32:05,4,64.99,1,tablet,2014-01-01 01:23:25,60091,2011-04-17 10:48:33,1994-07-18
290,78,4,2014-01-01 00:54:10,1,37.50,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18
379,457,27,2014-01-01 06:37:35,1,19.16,1,mobile,2014-01-01 06:34:20,60091,2011-04-17 10:48:33,1994-07-18
335,477,9,2014-01-01 02:30:35,3,41.70,1,desktop,2014-01-01 02:15:25,60091,2011-04-17 10:48:33,1994-07-18
293,103,4,2014-01-01 00:57:25,5,20.79,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18
271,390,22,2014-01-01 05:21:45,2,54.83,4,desktop,2014-01-01 05:21:45,60091,2011-04-08 20:08:14,2006-08-15
404,476,29,2014-01-01 07:24:10,4,121.59,1,mobile,2014-01-01 07:10:05,60091,2011-04-17 10:48:33,1994-07-18
179,90,3,2014-01-01 00:35:45,1,75.73,4,mobile,2014-01-01 00:28:10,60091,2011-04-08 20:08:14,2006-08-15


In [5]:
products_df = data["products"]
products_df

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [6]:
es = ft.EntitySet()

In [7]:
 es = es.entity_from_dataframe(entity_id="transactions",
                              dataframe=transactions_df,
                              index="transaction_id",
                              time_index="transaction_time",
                              variable_types={"product_id": ft.variable_types.Categorical,
                                              "zip_code": ft.variable_types.ZIPCode})

In [8]:
es

Entityset: None
  Entities:
    transactions [Rows: 500, Columns: 11]
  Relationships:
    No relationships

In [9]:
es["transactions"].variables

[<Variable: transaction_id (dtype = index)>,
 <Variable: session_id (dtype = numeric)>,
 <Variable: transaction_time (dtype: datetime_time_index, format: None)>,
 <Variable: amount (dtype = numeric)>,
 <Variable: customer_id (dtype = numeric)>,
 <Variable: device (dtype = categorical)>,
 <Variable: session_start (dtype: datetime, format: None)>,
 <Variable: join_date (dtype: datetime, format: None)>,
 <Variable: date_of_birth (dtype: datetime, format: None)>,
 <Variable: product_id (dtype = categorical)>,
 <Variable: zip_code (dtype = zipcode)>]

In [10]:
es = es.entity_from_dataframe(entity_id="products",
                             dataframe=products_df,
                              index="product_id")

In [11]:
es

Entityset: None
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    No relationships

In [12]:
new_relationship = ft.Relationship(es["products"]["product_id"],
                                  es["transactions"]["product_id"])

In [13]:
es = es.add_relationship(new_relationship)

In [14]:
es

Entityset: None
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    transactions.product_id -> products.product_id

In [15]:
es = es.normalize_entity(base_entity_id="transactions",
                         new_entity_id="sessions",
                         index="session_id",
                         make_time_index="session_start",
                         additional_variables=["device", "customer_id", "zip_code", "session_start", "join_date"])

In [16]:
es

Entityset: None
  Entities:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 6]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id

In [17]:
es["sessions"]

Entity: sessions
  Variables:
    session_id (dtype: index)
    device (dtype: categorical)
    customer_id (dtype: numeric)
    zip_code (dtype: zipcode)
    session_start (dtype: datetime_time_index)
    join_date (dtype: datetime)
  Shape:
    (Rows: 35, Columns: 6)

In [19]:
es["sessions"].variables

[<Variable: session_id (dtype = index)>,
 <Variable: device (dtype = categorical)>,
 <Variable: customer_id (dtype = numeric)>,
 <Variable: zip_code (dtype = zipcode)>,
 <Variable: session_start (dtype: datetime_time_index, format: None)>,
 <Variable: join_date (dtype: datetime, format: None)>]

In [18]:
es["transactions"].variables

[<Variable: transaction_id (dtype = index)>,
 <Variable: session_id (dtype = id)>,
 <Variable: transaction_time (dtype: datetime_time_index, format: None)>,
 <Variable: amount (dtype = numeric)>,
 <Variable: date_of_birth (dtype: datetime, format: None)>,
 <Variable: product_id (dtype = categorical)>]

In [20]:
es["transactions"].df.head(5)

,transaction_id,session_id,transaction_time,amount,date_of_birth,product_id
298,298,1,2014-01-01 00:00:00,127.64,1986-08-18,5
2,2,1,2014-01-01 00:01:05,109.48,1986-08-18,2
308,308,1,2014-01-01 00:02:10,95.06,1986-08-18,3
116,116,1,2014-01-01 00:03:15,78.92,1986-08-18,4
371,371,1,2014-01-01 00:04:20,31.54,1986-08-18,3


In [21]:
es = es.normalize_entity(base_entity_id="sessions",
                          new_entity_id="customers",
                          index="customer_id",
                          make_time_index="join_date",
                          additional_variables=["zip_code", "join_date"])

In [22]:
es

Entityset: None
  Entities:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 4]
    customers [Rows: 5, Columns: 3]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

In [23]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                    target_entity="products")

In [24]:
feature_defs

[<Feature: brand>,
 <Feature: SUM(transactions.amount)>,
 <Feature: STD(transactions.amount)>,
 <Feature: MAX(transactions.amount)>,
 <Feature: SKEW(transactions.amount)>,
 <Feature: MIN(transactions.amount)>,
 <Feature: MEAN(transactions.amount)>,
 <Feature: COUNT(transactions)>,
 <Feature: NUM_UNIQUE(transactions.session_id)>,
 <Feature: MODE(transactions.session_id)>,
 <Feature: NUM_UNIQUE(transactions.DAY(transaction_time))>,
 <Feature: NUM_UNIQUE(transactions.DAY(date_of_birth))>,
 <Feature: NUM_UNIQUE(transactions.YEAR(transaction_time))>,
 <Feature: NUM_UNIQUE(transactions.YEAR(date_of_birth))>,
 <Feature: NUM_UNIQUE(transactions.MONTH(transaction_time))>,
 <Feature: NUM_UNIQUE(transactions.MONTH(date_of_birth))>,
 <Feature: NUM_UNIQUE(transactions.WEEKDAY(transaction_time))>,
 <Feature: NUM_UNIQUE(transactions.WEEKDAY(date_of_birth))>,
 <Feature: NUM_UNIQUE(transactions.sessions.device)>,
 <Feature: NUM_UNIQUE(transactions.sessions.customer_id)>,
 <Feature: MODE(transactions.DA